In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch

# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torch.mps import empty_cache
import torch
from langchain.chains import RetrievalQA

torch.manual_seed(1234)


In [2]:

class Qwen:
    def __init__(self, model_path: str, device: str = "cuda:0"):
        self.model_path = model_path
        self.device = device
        self.tokenizer = None
        self.model = None

    def load_model(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_path, trust_remote_code=True)
        self.model.to(self.device)
        self.model.eval()
        self.model.generation_config = GenerationConfig.from_pretrained(self.model_path, trust_remote_code=True)

    def generate_response(self, image_folder: str, image_index: int, prompt: str):
        query = self.tokenizer.from_list_format([
            {'image': f'{image_folder}/{image_index}.png'}, 
            {'text': prompt}
        ])

        # Ensure tensors are on the correct device if necessary
        if isinstance(query, torch.Tensor):
            query = query.to(self.device)

        response, history = self.model.chat(self.tokenizer, query=query, history=None)
        return response

# Path to the model directory
model_path = "/data1/dxw_data/llm/Qwen-VL-Chat"
# Specify the device (e.g., 'cuda:0', 'cuda:1')
device = 'cuda:0'

# Instantiate and load the model
qwen_model = Qwen(model_path, device)
qwen_model.load_model()

2024-06-15 09:03:13.678505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-15 09:03:13.797971: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 09:03:14.367473: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-06-15 09:03:14.367746: W tensorflow/compiler/xla/stream_exec

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [3]:

# Example usage
image_folder = "/data1/dxw_data/llm/mkt-englishwords/data"
image_index = 0  # Change this to the actual image index
prompt = "Generate some descriptions of the clothes in this image, following the social media style of tone"

response = qwen_model.generate_response(image_folder, image_index, prompt)
print(response)

The woman in the image is wearing a beautiful pastel pink skirt that flows beautifully when she stands on the stairs. She also wears a simple white top that complements the skirt perfectly. The outfit is completed with a pair of comfortable beige sandals. The overall style is very suitable for a casual summer outing with friends.
